In [1]:
!nsys profile -s none --stats=true python3 icassp_scipy_v1.py float32 2

Processing events...
Saving temporary "/tmp/nsys-report-04c2-865c-33de-b9ff.qdstrm" file to disk...

Creating final output files...
Processing [==============================================================100%]
Saved report file to "/tmp/nsys-report-04c2-865c-33de-b9ff.qdrep"
Exporting 2140 events: [==================================================100%]

Exported successfully to
/tmp/nsys-report-04c2-865c-33de-b9ff.sqlite


Operating System Runtime API Statistics:

 Time(%)  Total Time (ns)  Num Calls   Average   Minimum   Maximum      Name   
 -------  ---------------  ---------  ---------  -------  ----------  ---------
    83.4       18,844,868        521   36,170.6    1,000  15,726,707  read     
     7.2        1,628,969        722    2,256.2    1,578       5,113  open64   
     3.8          853,171          1  853,171.0  853,171     853,171  fork     
     2.1          481,060        158    3,044.7    1,016      23,797  munmap   
     2.0          445,925          1  445,925.0 

In [2]:
!nsys profile -s none --stats=true python3 icassp_numba_v1.py float32 2

Processing events...
Saving temporary "/tmp/nsys-report-ee93-4763-34ec-b03b.qdstrm" file to disk...

Creating final output files...
Processing [==============================================================100%]
Saved report file to "/tmp/nsys-report-ee93-4763-34ec-b03b.qdrep"
Exporting 4178 events: [==================================================100%]

Exported successfully to
/tmp/nsys-report-ee93-4763-34ec-b03b.sqlite


CUDA API Statistics:

 Time(%)  Total Time (ns)  Num Calls    Average      Minimum     Maximum           Name       
 -------  ---------------  ---------  ------------  ----------  ----------  ------------------
    81.6       37,361,902          3  12,453,967.3  11,802,371  12,874,623  cuCtxSynchronize  
     8.7        3,981,134          3   1,327,044.7       9,497   2,543,624  cuMemcpyHtoD_v2   
     4.5        2,078,941          1   2,078,941.0   2,078,941   2,078,941  cuModuleLoadDataEx
     3.1        1,438,488          1   1,438,488.0   1,438,488   1,438,48

In [3]:
!nsys profile -s none --stats=true python3 icassp_numba_v2.py float32 2

Registers 60
Processing events...
Saving temporary "/tmp/nsys-report-dff9-4c5d-fda0-6a06.qdstrm" file to disk...

Creating final output files...
Processing [==============================================================100%]
Saved report file to "/tmp/nsys-report-dff9-4c5d-fda0-6a06.qdrep"
Exporting 4153 events: [==================================================100%]

Exported successfully to
/tmp/nsys-report-dff9-4c5d-fda0-6a06.sqlite


CUDA API Statistics:

 Time(%)  Total Time (ns)  Num Calls    Average      Minimum     Maximum           Name       
 -------  ---------------  ---------  ------------  ----------  ----------  ------------------
    84.7       39,653,461          3  13,217,820.3  11,519,421  15,307,030  cuCtxSynchronize  
     7.6        3,553,186          3   1,184,395.3      11,237   2,242,165  cuMemcpyHtoD_v2   
     3.3        1,560,973          1   1,560,973.0   1,560,973   1,560,973  cuMemcpyDtoH_v2   
     2.2        1,043,755          1   1,043,755.0   1,043,7

In [4]:
!nsys profile -s none --stats=true python3 icassp_numba_v3.py float32 2

Registers 42
Processing events...
Saving temporary "/tmp/nsys-report-cc49-3e72-0ee4-97df.qdstrm" file to disk...

Creating final output files...
Processing [==============================================================100%]
Saved report file to "/tmp/nsys-report-cc49-3e72-0ee4-97df.qdrep"
Exporting 4218 events: [==================================================100%]

Exported successfully to
/tmp/nsys-report-cc49-3e72-0ee4-97df.sqlite


CUDA API Statistics:

 Time(%)  Total Time (ns)  Num Calls    Average     Minimum    Maximum          Name       
 -------  ---------------  ---------  -----------  ---------  ---------  ------------------
    46.3        6,923,029          3  2,307,676.3  1,895,723  2,620,882  cuCtxSynchronize  
    24.3        3,634,717          3  1,211,572.3     10,465  2,243,837  cuMemcpyHtoD_v2   
    12.1        1,812,720          1  1,812,720.0  1,812,720  1,812,720  cuModuleLoadDataEx
     9.9        1,481,939          1  1,481,939.0  1,481,939  1,481,939  cu

In [5]:
!nsys profile -s none --stats=true python3 icassp_numba_v4.py float32 2

Registers 38
Processing events...
Saving temporary "/tmp/nsys-report-6478-d434-3fc6-a71a.qdstrm" file to disk...

Creating final output files...
Processing [==============================================================100%]
Saved report file to "/tmp/nsys-report-6478-d434-3fc6-a71a.qdrep"
Exporting 4225 events: [==================================================100%]

Exported successfully to
/tmp/nsys-report-6478-d434-3fc6-a71a.sqlite


CUDA API Statistics:

 Time(%)  Total Time (ns)  Num Calls    Average     Minimum    Maximum          Name       
 -------  ---------------  ---------  -----------  ---------  ---------  ------------------
    44.1        6,022,629          3  2,007,543.0  1,006,542  2,660,974  cuCtxSynchronize  
    26.6        3,624,628          3  1,208,209.3     10,371  2,211,129  cuMemcpyHtoD_v2   
    13.8        1,882,044          1  1,882,044.0  1,882,044  1,882,044  cuModuleLoadDataEx
     8.9        1,216,382          1  1,216,382.0  1,216,382  1,216,382  cu

In [6]:
!nsys profile -s none --stats=true python3 icassp_numba_v5.py float32 2

Registers 32
Processing events...
Saving temporary "/tmp/nsys-report-5902-905a-253b-ae7a.qdstrm" file to disk...

Creating final output files...
Processing [==============================================================100%]
Saved report file to "/tmp/nsys-report-5902-905a-253b-ae7a.qdrep"
Exporting 4221 events: [==================================================100%]

Exported successfully to
/tmp/nsys-report-5902-905a-253b-ae7a.sqlite


CUDA API Statistics:

 Time(%)  Total Time (ns)  Num Calls    Average     Minimum    Maximum          Name       
 -------  ---------------  ---------  -----------  ---------  ---------  ------------------
    51.7        7,294,897          3  2,431,632.3  2,200,222  2,704,479  cuCtxSynchronize  
    25.8        3,642,792          3  1,214,264.0     10,586  2,208,768  cuMemcpyHtoD_v2   
     8.8        1,235,260          1  1,235,260.0  1,235,260  1,235,260  cuMemcpyDtoH_v2   
     7.6        1,065,201          1  1,065,201.0  1,065,201  1,065,201  cu